In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import random

import os
os.environ["PROJ_LIB"] = "C:\\Users\\gabri\\AppData\\Local\\Continuum\\anaconda3\\pkgs\\proj4-5.2.0-ha925a31_1\\Library\\share";
from mpl_toolkits.basemap import Basemap

import sys
sys.path.append("../../../..")
from definitions import QUAKES_RAW_DATA_PATH
import networkx as nx

from earthquakes.quakes_util import custom_geo_distance, is_point_in_bbox, get_earthquakes_in_timespan, find_closest_station, UTCDateTime_to_string, \
    string_to_UTCDateTime, quake_filename_to_UTCDateTime, BadDataException, read_pickle, write_pickle, obtain_data, timeit_context, visualize_sensors_graph, create_weighted_adjacency_quakes, \
    visualize_quakes, resample_data, k_hop_reach, recalls_at_k_hops, precisions_at_k_hops, f1_scores_at_k_hops, compute_metrics_with_growing_radius
from utils.misc_utils import check_create_folder

from prediction.train_utils import plot_cm


# taken from https://www.openstreetmap.org/export
bounding_box = [
    166.104, # left lower corner longitude
    178.990, # upper right corner longitude
    -47.749, # left lower corner latitude
    -33.779, # upper right corner latitude
]

nz_basemap = Basemap(
        projection='merc',
        llcrnrlon=bounding_box[0],  # left lower corner longitude
        llcrnrlat=bounding_box[2],  # left lower corner latitude
        urcrnrlon=bounding_box[1],
        urcrnrlat=bounding_box[3],
        lat_ts=0,
        resolution='c',
        suppress_ticks=True)

C:\Users\gabri\AppData\Local\Continuum\anaconda3\envs\tud_ml4se\lib\site-packages\ipykernel_launcher.py:40: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.


In [2]:
NEW_DATASET_FOLDER = os.path.join(QUAKES_RAW_DATA_PATH, "new_dataset_gabriele")
stations_in_bbox = pd.read_csv(os.path.join(NEW_DATASET_FOLDER, "stations_in_bbox.csv"))
weighted_adjacency = np.load(os.path.join(NEW_DATASET_FOLDER, "packaged", "weighted_adjacency.npy"))
distances_matrix = np.load(os.path.join(NEW_DATASET_FOLDER, "packaged", "pairwise_distances.npy"))
labels = np.load(os.path.join(NEW_DATASET_FOLDER, "packaged", "labels.npy"))
class_to_station_dict = read_pickle(os.path.join(NEW_DATASET_FOLDER, "packaged", "class2label_dict.pickle"))

print(stations_in_bbox.shape)
stations_in_bbox.head()

(58, 9)


Unnamed: 0 #network station   latitude   longitude  elevation  \
0           0       NZ     BFZ -40.679647  176.246245      283.0   
1           1       NZ     BHW -41.408231  174.871115      101.0   
2           2       NZ     BKZ -39.165666  176.492544      706.0   
3           3       NZ    COVZ -39.199914  175.542402     1133.0   
4           4       NZ     CVZ -44.383180  171.006160      454.0   

              sitename            starttime  endtime  
0           Birch Farm  1996-04-03T00:00:00      NaN  
1          Baring Head  1975-11-27T00:00:00      NaN  
2     Black Stump Farm  2004-02-11T00:00:00      NaN  
3  Chateau Observatory  1998-03-25T00:00:00      NaN  
4                 Cave  2014-05-16T00:00:00      NaN

In [3]:
station_lats=list(stations_in_bbox.latitude.values)
station_lons=list(stations_in_bbox.longitude.values)
station_coords = zip(station_lons, station_lats)


In [8]:
def map_station_to_region(lonlat):
    bb_region_0 = [165.498, 176.285, -48.472 ,-43.3] #[166.104, 173.210, -47.749, -43.992]
    bb_region_1 = [168.399, 174.441, -43.3, -40.279]
    bb_region_2 = [173.584, 179.231, -41.904, -38.754] #-38.754
    bb_region_3 = [171.496, 178.901, -38.754, -33.599]
    station_region = None
    for idx, bbox in enumerate([bb_region_0, bb_region_1, bb_region_2, bb_region_3]):
        if is_point_in_bbox(lonlat, bbox):
            if station_region is not None:
                raise Exception
            else:
                station_region = idx
    return station_region

stations_in_bbox['region'] = stations_in_bbox.apply(lambda row: map_station_to_region((row.longitude, row.latitude)), axis=1)
stations_in_bbox.head()

stations_in_bbox.to_csv(os.path.join(NEW_DATASET_FOLDER, "stations_in_bbox.csv"), index=False)


colors = ['white', 'red', 'green', 'yellow']

nodecolors = [colors[region_index] for region_index in stations_in_bbox.region]


plt.figure(figsize=(8, 8))
visualize_sensors_graph(station_lons, station_lats, weighted_adjacency, 
                        nz_basemap, nodesize=200, nodecolor=nodecolors, 
                        draw_edges=True, edgecolors='white', with_labels=True,
                       alpha=1)
plt.tight_layout()
plt.savefig("division_into_regions.png")
plt.show()

In [5]:
mapping_dict_4_regions = {station_idx: region_idx for station_idx, region_idx in enumerate(stations_in_bbox.region)}
unique, counts = np.unique(list(mapping_dict_4_regions.values()), return_counts=True)
print(unique)
print(f"Counts of stations per region: {list(counts)}")

[0 1 2 3]
Counts of stations per region: [16, 9, 15, 18]


# Analysis of how many earthquakes each region gets

In [6]:
labels.shape

(4633,)

In [7]:
new_labels = np.zeros(labels.shape)
regions = stations_in_bbox.region
for idx, class_label in enumerate(labels):
    corresponding_station = class_to_station_dict[class_label]
    new_labels[idx] = regions[corresponding_station]


In [8]:
bars, heights = np.unique(new_labels, return_counts=True)
plt.bar(bars, heights)
plt.show()

# Clipping (to make it balanced)

In [9]:
num_of_desired_samples_per_class = min(heights)
num_of_desired_samples_per_class

877

In [10]:
classes_indices = {}
indices = list(range(new_labels.shape[0]))
for i in indices:
    label = new_labels[i]
    classes_indices[label] = classes_indices.get(label, []) + [i]

for k,v in classes_indices.items():
    print(f"{k} --> {len(v)} values")
    
# Now we clip the classes which have more values than the minimum one
print("\n")

for k,v in classes_indices.items():
    classes_indices[k] = random.sample(classes_indices[k], num_of_desired_samples_per_class)
    
all_values = []
for k,v in classes_indices.items():
    print(f"{k} --> {len(v)} values")
    all_values += v
assert len(np.unique(all_values)) == num_of_desired_samples_per_class*len(classes_indices.keys())

2.0 --> 1428 values
0.0 --> 877 values
1.0 --> 978 values
3.0 --> 1350 values


2.0 --> 877 values
0.0 --> 877 values
1.0 --> 877 values
3.0 --> 877 values


In [11]:
original_data = np.load(os.path.join(NEW_DATASET_FOLDER, "packaged", "data_downsampledAt2Hz.npy"))
print(original_data.shape)
labels_4_regions = new_labels
assert original_data.shape[0] == labels_4_regions.shape[0]
assert len(np.unique(labels_4_regions)) == 4

(4633, 58, 120)


In [12]:
print(f"The final balanced data will have {len(all_values)} data points ({num_of_desired_samples_per_class} for each of the {len(classes_indices.keys())} classes).")

The final balanced data will have 3508 data points (877 for each of the 4 classes).


In [13]:
n_regions = len(classes_indices.keys())
print(f"{n_regions} regions.\n")


indices = []
final_labels = []
for region_label in classes_indices:
    indices_for_region = classes_indices[region_label]
    indices += indices_for_region
    for _ in indices_for_region:
        final_labels.append(region_label)
    
print(len(indices), len(final_labels))

final_data = np.take(original_data, indices, axis=0)
final_labels = np.array(final_labels)
print(final_data.shape, final_labels.shape)
assert final_data.shape[0] == final_labels.shape[0]
assert len(np.unique(final_data, axis=0)) == final_data.shape[0]
assert len(np.unique(final_labels, axis=0)) == n_regions

bars, heights = np.unique(final_labels, return_counts=True)
plt.bar(bars, heights)
plt.show()

4 regions.

3508 3508
(3508, 58, 120) (3508,)


In [15]:
balanced_dataset_folder = os.path.join(NEW_DATASET_FOLDER, "packaged", "4regions_balanced")
check_create_folder(balanced_dataset_folder)

In [16]:
#np.save(os.path.join(balanced_dataset_folder, "data_downsampledAt2Hz.npy"), final_data)
#np.save(os.path.join(balanced_dataset_folder, "labels.npy"), final_labels)